# Fisher's exact test from scratch with Python:


In [1]:
import numpy as np # import libraries
import pandas as pd
import scipy
import scipy.special

In [2]:
# create contingency table with floats to avoid datatype issues with pd.DataFrame.at 
ar=np.array([[7.0, 2.0],[1.0, 12.0]])    
df=pd.DataFrame(ar, columns=["Responded", "No Response"])
df.index=["Males", "Females"] 
df 

,Responded,No Response
Males,7.0,2.0
Females,1.0,12.0


In [3]:
df2=df.copy() # create contingency table with the marginal totals and the grand total. 
df2.loc['Column_Total']= df2.sum(numeric_only=True, axis=0)
df2.loc[:,'Row_Total'] = df2.sum(numeric_only=True, axis=1)
df2

,Responded,No Response,Row_Total
Males,7.0,2.0,9.0
Females,1.0,12.0,13.0
Column_Total,8.0,14.0,22.0


In [4]:
n=df2.at["Column_Total", "Row_Total"]  # grand total 

exp=df2.copy()               # create dataframe with expected frequencies
for x in exp.index[0:-1]:
    for y in exp.columns[0:-1]:
        # round expected values to nearest hundredths:
        v= (((df2.at[x, "Row_Total"]) * (df2.at["Column_Total", y])   )   /n ).round(2) 
        exp.at[x,y]=float(v)

exp

,Responded,No Response,Row_Total
Males,3.27,5.73,9.0
Females,4.73,8.27,13.0
Column_Total,8.00,14.00,22.0


In [5]:
# create function that takes the upper left cell of a contingency table as input and 
# returns the probability to observe this particular contingency table.

def p(a): 
    v=(scipy.special.binom(int(df2.iloc[0,2]), a) * scipy.special.binom(int(df2.iloc[1,2]), (int(df2.iloc[2,0])-a)) )/scipy.special.binom(n, int(df2.iloc[2,0]))
    return v

p(1) # if we try "a=1" we get the following probability ... 

0.04829721362229102

In [6]:
p_observed=p(7) # In our contingency table, a was equal to 7.

p_list=[]
for i in range(int(df2.iloc[0,2])  + 1  ): # calculate p(a) for every possible table we can get given the fixed margins...
#... this ranges from "9 choose 0" to "9 choose 9" so we should get 10 possible tables and their respective probabilities..
    if p(i)<=p_observed:
        p_list.append(p(i))     # append these probabilites to p_list only if <= p_observed
        
p_val=np.sum(p_list) # the sum of this list corresponds to the p-value 
p_val 

0.0014916971573318324

In [7]:
import scipy.stats as stats

oddsratio, pvalue = stats.fisher_exact([[7, 2],[1, 12]])   
pvalue

0.0014916971573318315